# Text-GAN Turkish word generation

In [1]:
using Knet, Test, Base.Iterators, Printf, LinearAlgebra, CuArrays, Random, IterTools, StatsBase

struct Charset
    c2i::Dict{Any,Int}
    i2c::Vector{Any}
    eow::Int
end

function Charset(charset::String; eow="")
    i2c = [ eow; [ c for c in charset ]  ]
    print(i2c)
    c2i = Dict( c => i for (i, c) in enumerate(i2c))
    return Charset(c2i, i2c, c2i[eow])
end

struct TextReader
    file::String
    charset::Charset
end

function Base.iterate(r::TextReader, s=nothing)
    s === nothing && (s = open(r.file))
    eof(s) && return close(s)
    return [ get(r.charset.c2i, c, r.charset.eow) for c in readline(s)], s
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

struct WordsData
    src::TextReader        
    batchsize::Int         
    maxlength::Int         
    batchmajor::Bool       
    bucketwidth::Int    
    buckets::Vector        
end

function WordsData(src::TextReader; batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 2, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar length
    WordsData(src, batchsize, maxlength, batchmajor, bucketwidth, buckets)
end

Base.IteratorSize(::Type{WordsData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{WordsData}) = Base.HasEltype()
Base.eltype(::Type{WordsData}) = NTuple{2}

function Base.iterate(d::WordsData, state=nothing)
    if state == 0 # When file is finished but buckets are partially full 
        for i in 1:length(d.buckets)
            if length(d.buckets[i]) > 0
                buc = d.buckets[i]
                d.buckets[i] = []
                return buc, state
            end
        end
        return nothing # Finish iteration
    end

    while true
        src_next = iterate(d.src, state)
        
        if src_next === nothing
            state = 0
            return iterate(d, state)
        end
        
        (src_word, src_state) = src_next
        state = src_state
        src_length = length(src_word)
        
        (src_length > d.maxlength) && continue

        i = Int(ceil(src_length / d.bucketwidth))
        i > length(d.buckets) && (i = length(d.buckets))

        push!(d.buckets[i], src_word)
        if length(d.buckets[i]) == d.batchsize
            buc = d.buckets[i]
            d.buckets[i] = []
            return buc, state
        end
    end
end

function readwordset(fname)
    words = []
    fi = open(fname)
    while !eof(fi)
        push!(words, readline(fi))
    end
    close(fi)
    words
end

readwordset (generic function with 1 method)

### G/D Common Parts

In [2]:
struct Embed; w; end

function Embed(shape...)
    Embed(param(shape...))
end

# per-word loss (in this case per-batch loss)
function loss(model, data; average=true)
    l = 0
    n = 0
    a = 0
    for (x, y) in data
        v = model(x, y; average=false)
        l += v[1]
        n += v[2]
        a += (v[1] / v[2])
    end
    average && return a
    return l, n
end

loss (generic function with 1 method)

## CNN Discriminator

In [3]:
# This one to be used by DModel, takes weights of characters and reduce the embedding for each character
# this approach to avoid sampling or argmaxing over rnn's output
# (C, B, T) -> (T, E, 1, B)

# function (l::Embed)(x)
#     dims = size(x)
#     em = l.w * reshape(x, dims[1], dims[2] * dims[3]) # reshape for multiplication 
#     em = reshape(em, size(em, 1), dims[2], dims[3]) # reshape to original size
#     em = permutedims(em, [3, 1, 2])  # permute for CONV
#     em = reshape(em, dims[3], size(em, 2), 1, dims[2]) # Add one dim for CONV
# end

# struct Conv; w; b; f; p; end
# (c::Conv)(x) = (co=conv4(c.w, dropout(x,c.p)); c.f.(pool((co .+ c.b); window=(size(co, 1), size(co, 2)))))
# Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

# struct Dense; w; b; f; p; end
# (d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
# Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

# # Perform convolution then, global-max pooling and concatenate the output and feed it to sequential dense layer 
# mutable struct DisModel
#     charset::Charset
#     embed::Embed
#     filters
#     dense_layers
# end

# # This discriminator uses separate weights for its embedding layer
# function DisModel(charset, embeddingSize::Int, filters, denselayers)
#     Em = Embed(embeddingSize, length(tr_charset.c2i))
#     DisModel(charset, Em, filters, denselayers)
# end

# function (c::DisModel)(x) # the input here is weights of the characters with shape (C, B, T)
#     em = c.embed(x)
#     filters_out = []
#     for f in c.filters
#         push!(filters_out, f(em))
#     end
#     max_out = cat(filters_out...;dims=3)
#     for l in c.dense_layers
#         max_out = l(max_out)
#     end
#     max_out
# end

# (c::DisModel)(x,y; average=true) = nll(c(x), y; average=average)


## Recurrent Discriminator

In [12]:
function (l::Embed)(x)
    dims = size(x)
    em = l.w * reshape(x, dims[1], dims[2] * dims[3]) # reshape for multiplication 
    em = reshape(em, size(em, 1), dims[2], dims[3]) # reshape to original size
end

struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 3-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

mutable struct DisModel
    charset::Charset
    embed::Embed
    rnn::RNN
    denselayers
end

# This discriminator uses separate weights for its embedding layer
function DisModel(charset, embeddingSize::Int, hidden, denselayers; layers=1, dropout=0)
    Em = Embed(embeddingSize, length(tr_charset.c2i))
    rnn = RNN(embeddingSize, hidden; numLayers=layers, dropout=dropout)
    DisModel(charset, Em, rnn, denselayers)
end

function (c::DisModel)(x) # the input here is weights of the characters with shape (C, B, T)
    c.rnn.h, c.rnn.c = 0, 0
    em = c.embed(x)
    rnn_out = permutedims(c.rnn(em), [1, 3, 2])
    for l in c.denselayers
        rnn_out = l(rnn_out)
    end
    rnn_out
end

(c::DisModel)(x,y; average=true) = nll(c(x), y; average=average)

## Generator

In [5]:
get_z(shape...) = KnetArray(randn(Float32, shape...))

### Not used 
# concatenate z with embedding vectors, z -> (z_size, B), returns (E+z_size, B, T)
# this will be used to feed Z to generator at each timestep
# function (l::Embed)(x, z)
#     em = l.w[:, x]
#     z_array = cat((z for i in 1:size(em, 3))...; dims=(3))
#     cat(em, z_array; dims=(1))
# end

# Generator model
struct GenModel
    projection::Embed
    rnn::RNN        
    dropout::Real
    charset::Charset
    disModel::DisModel
    maxlength::Int
end

function GenModel(inputsize::Int, hidden::Int, charset::Charset, disModel::DisModel, maxlength::Int; layers=2, dropout=0)
    rnn = RNN(inputsize, hidden; numLayers=layers, dropout=dropout)
    projection = Embed(hidden, length(charset.i2c))
    GenModel(projection, rnn, dropout, charset, disModel, maxlength)
end

# This generator shares the projection layers weights of the discriminator for its projection layer
function GenModel(inputsize::Int, charset::Charset, disModel::DisModel, maxlength::Int; layers=2, dropout=0)
    rnn = RNN(inputsize, size(disModel.embed.w, 1); numLayers=layers, dropout=dropout)
    GenModel(disModel.embed, rnn, dropout, charset, disModel, maxlength)
end

function Z(s::GenModel, batchsize, timesteps)
    z = get_z(s.rnn.inputSize, batchsize, 1) # according to get_z(H, B, layers)
    return cat([ z for i in 1:timesteps]...;dims=3)
end

# Generator forward pass, size(Z) -> inputsize, batchsize, sequencelength
function (s::GenModel)(Z)
    s.rnn.h, s.rnn.c = 0, 0
    rnn_out = s.rnn(Z) 
    dims = size(rnn_out)
    output = s.projection.w' * dropout(reshape(rnn_out, dims[1], dims[2] * dims[3]), s.dropout)
    reshape(softmax(output), size(output, 1), dims[2], dims[3])
end

# Generator loss
function (s::GenModel)(Z, calculateloss::Int; average=true)
    y = Array(ones(Int, size(Z, 2))) # create labels 1 -> real, 2-> not-real
    x = s(Z)
    pads = KnetArray(zeros(Float32, size(x, 1), size(x, 2), s.maxlength - size(x, 3)))
    pads[s.charset.eow, :, :] .= 1
    x = cat(x, pads; dims=3) # padding
    return s.disModel(x, y;average=average) 
end

function generate(s::GenModel, maxlength, batchsize)
    out = s(Z(s, batchsize, maxlength))
    words = []
    for i in 1:batchsize
        push!(words, join([s.charset.i2c[x[1]] for x in argmax(out[:, i, :]; dims=1)], ""))
    end
    words
end

generate (generic function with 1 method)

## Word Sampler

Word sampler will be used to train discriminator.
this sampler should take B, T (batchsize, timestep) as parameters
returns (X, Y) tuple 
where X is tensor of size (C, B, T)
and Y is array of size B
B consists of real words and generated words
C charset size where each value is weight of this char
in the case of generated words the generator already gives C, B, T
for real words we need to convert words to C, T arrays
where every character can be represented by one hot vector or by Gumble-Max (which is normalized one hot vector)

In [6]:
struct Sampler
    wordsdata::WordsData
    charset::Charset
    genModel::GenModel
    maxBatchsize::Int
end

# this function is similar to gumble softmax, it is used to soften the one-hot-vector of the real samples
# tau -> normalization factor; the bigger the softer
function soften(A; dims=1, tau=2.0) 
    A = A ./ tau
    softmax(A; dims=dims)
end

Base.IteratorSize(::Type{Sampler}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{Sampler}) = Base.HasEltype()
Base.eltype(::Type{Sampler}) = Tuple{KnetArray{Float32,3},Array{Int64,1}}

function Base.iterate(s::Sampler, state=nothing)
    wdatastate = iterate(s.wordsdata, state)
    wdatastate === nothing && (return nothing)
    
    (bucket, state) = wdatastate
    bsize = length(bucket)
    src_eow = s.charset.eow
    src_lengths = map(x -> length(x), bucket)
    max_length = max(src_lengths...)
    gsize = 1 + rand(bsize:s.maxBatchsize) - bsize # count of words to be generated
    generated = s.genModel(Z(s.genModel, gsize, max_length))

    to_be_cat = [generated, ]
    for (i, v) in enumerate(bucket)
        tindex = [i for i in 1:length(v)]
        onehot = KnetArray(zeros(Float32, length(s.charset.c2i), 1, max_length))
        onehot[v, :, tindex] .= 1
        onehot = soften(onehot) # soften one hot vectors elements value
        push!(to_be_cat, onehot)
    end
    x = cat(to_be_cat...;dims=2) # concatenate both generated and sampled words
    pads = KnetArray(zeros(Float32, size(x, 1), size(x, 2), s.wordsdata.maxlength - size(x, 3)))
    pads[s.charset.eow, :, :] .= 1
    
#     pads = soften(pads) # getting error 

    x = cat(x, pads; dims=3) # padding
    y = Array(ones(Int, gsize+bsize)) # create labels 1 -> real, 2-> not-real
    y[1:gsize] = y[1:gsize] .+ 1
    
    ind = shuffle(1:gsize+bsize) # used to shuffle the batch
    x, y = x[:, ind, :], y[ind]
    return (x,y), state
end

In [7]:
function train!(model, parameters, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn; params=parameters), seconds=30) do y
        devloss = loss(model, dev)
        tstloss = map(d->loss(model,d), tst)
        if devloss < bestloss
            bestloss, bestmodel = devloss, deepcopy(model)
        end
        println(stderr)
        (dev=devloss, tst=tstloss, mem=Float32(CuArrays.usage[]))
    end
    return bestmodel
end

train! (generic function with 1 method)

In [8]:
# per-word loss (in this case per-batch loss)
function loss(model, data; average=true)
    l = 0
    n = 0
    a = 0
    for (x, y) in data
        v = model(x, y; average=false)
        l += v[1]
        n += v[2]
        a += (v[1] / v[2])
    end
    average && return a
    return l, n
end

loss (generic function with 1 method)

In [24]:
char_set = "ABCDEFGHIJKLMNOPRSTUVYZabcdefghijklmnoprstuvyzÇÖÜçöüĞğİıŞş"
tr_charset = Charset(char_set)
datadir = "turkish_word_set"
BATCHSIZE = 128
MAXLENGTH = 7
tr_dev = TextReader("$datadir/dev.tr", tr_charset)
tr_trn = TextReader("$datadir/train.tr", tr_charset)
dtrn = WordsData(tr_trn, batchsize=BATCHSIZE, maxlength=MAXLENGTH, bucketwidth = 1)
ddev = WordsData(tr_dev, batchsize=BATCHSIZE, maxlength=MAXLENGTH, bucketwidth = 1)

Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'Ç', 'Ö', 'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş']

WordsData(TextReader("turkish_word_set/dev.tr", Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1)), 128, 7, false, 1, Array{Any,1}[[], [], [], [], [], [], []])

In [28]:
EMBEDDING_SIZE = 64
DHIDDEN_SIZE = 128
GDROPOUT = 0.2
DDROPOUT = 0.2

dismodel = DisModel(tr_charset, EMBEDDING_SIZE, DHIDDEN_SIZE,(
        Dense(DHIDDEN_SIZE * MAXLENGTH, 16, pdrop=DDROPOUT),
        Dense(16, 2, sigm, pdrop=0.2)
        ); dropout=DDROPOUT)

GE_SIZE = 256
Z_SIZE = 128

genmodel = GenModel(Z_SIZE, GE_SIZE, tr_charset, dismodel, MAXLENGTH; dropout=GDROPOUT, layers=2)
trnsampler = Sampler(dtrn, tr_charset, genmodel, BATCHSIZE * 2)
devsampler = Sampler(ddev, tr_charset, genmodel, BATCHSIZE * 2)

Sampler(WordsData(TextReader("turkish_word_set/dev.tr", Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1)), 128, 7, false, 1, Array{Any,1}[[], [], [], [], [], [], []]), Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1), GenModel(Embed(P(KnetArray{Float32,2}(256,59))), LSTM(input=128,hidden=256,layers=2,dropout=0.2), 0.2, Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1), DisModel(Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24

In [29]:
function gmodel(epochs)
    global genmodel
    global BATCHSIZE
    global MAXLENGTH
    
    ctrn = [ (Z(genmodel, BATCHSIZE, MAXLENGTH), 1) for i in 1:500 ]
    trnxepoch = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
    trnmini = ctrn[1:20]
    dev = [ (Z(genmodel, BATCHSIZE, MAXLENGTH), 1) for i in 1:100 ]
    genmodel = train!(genmodel, params(genmodel)[1:2], trnxepoch, dev, trnmini)
end

function dmodel(batches)
    global trnsampler
    global devsampler
    global dismodel
    
    ctrn = collect(trnsampler)
    trnmini = ctrn[1:20]
    ctrn = shuffle!(ctrn)[1:batches]
    dev = collect(devsampler)
    dismodel = train!(dismodel, params(dismodel), ctrn, dev, trnmini) 
end

@info "Started training..."
for k in 1:20
    println("Turn no:", k)
    println("Ex.Generated words: ", join(generate(genmodel, MAXLENGTH, 5),"\n"))

    println("Training Discriminator:")
    dmodel(Int(ceil(rand() * 30)))
    println("Training Generator:")
    gmodel(Int(ceil(rand() * 10)))
end

┌ Info: Started training...
└ @ Main In[29]:25


Turn no:1
Ex.Generated words: VVVVVVV
ÖÖÖkkkk
ttççççç
İİbbbbb
vvvvvİİ
Training Generator:



┣██                  ┫ [10.00%, 1/10, 00:00/00:01, 10.64i/s] (dev = 50.550514f0, tst = (13.850113f0,), mem = 2.8813647f9)
┣████████████████████┫ [100.00%, 10/10, 00:00/00:00, 65.72i/s] (dev = 49.052547f0, tst = (13.487146f0,), mem = 2.8834696f9)

┣                    ┫ [0.10%, 1/1000, 00:00/03:56, 4.24i/s] (dev = 66.78007f0, tst = (13.356004f0,), mem = 7.971089f9)
┣████████████████████┫ [100.00%, 1000/1000, 00:05/00:05, 194.71i/s] (dev = 66.772545f0, tst = (13.354519f0,), mem = 1.3836291f10)


Training Discriminator:
Turn no:2
Ex.Generated words: sssss
sssss
sssss
sssss
sssss
Training Generator:



┣█▊                  ┫ [9.09%, 1/11, 00:00/00:01, 19.76i/s] (dev = 48.73716f0, tst = (13.2623825f0,), mem = 1.3836448f10)
┣███████████████████▉┫ [100.00%, 11/11, 00:00/00:00, 100.53i/s] (dev = 42.06518f0, tst = (11.030857f0,), mem = 1.3836448f10)

┣                    ┫ [0.03%, 1/3000, 00:00/06:59, 7.15i/s] (dev = 66.772545f0, tst = (13.354521f0,), mem = 1.3836448f10)
┣████████████████████┫ [100.00%, 3000/3000, 00:13/00:13, 230.44i/s] (dev = 66.772545f0, tst = (13.35452f0,), mem = 1.3937111f10)


...

Training Discriminator:
Turn no:19
Ex.Generated words: sssss
sssssss
ssşşttt
sssss
sssss
Training Generator:



┣████                ┫ [20.00%, 1/5, 00:00/00:00, 16.48i/s] (dev = 22.868233f0, tst = (6.2652693f0,), mem = 1.1633609f10)
┣████████████████████┫ [100.00%, 5/5, 00:00/00:00, 47.24i/s] (dev = 22.868221f0, tst = (6.265267f0,), mem = 1.1633609f10)

┣                    ┫ [0.03%, 1/4000, 00:00/09:23, 7.10i/s] (dev = 66.77252f0, tst = (13.354513f0,), mem = 1.1633609f10)
┣████████████████████┫ [100.00%, 4000/4000, 00:17/00:17, 232.66i/s] (dev = 66.77252f0, tst = (13.354514f0,), mem = 1.170809f10)


Training Discriminator:
Turn no:20
Ex.Generated words: sssssss
ssssss
ssCCCCC
ssssss
ssşjjj
Training Generator:



┣█▊                  ┫ [9.09%, 1/11, 00:00/00:01, 19.15i/s] (dev = 22.868221f0, tst = (6.2652655f0,), mem = 1.1719692f10)
┣███████████████████▉┫ [100.00%, 11/11, 00:00/00:00, 99.57i/s] (dev = 22.86821f0, tst = (6.2652626f0,), mem = 1.1719692f10)

┣                    ┫ [0.03%, 1/3000, 00:00/06:45, 7.41i/s] (dev = 66.77253f0, tst = (13.354513f0,), mem = 1.1719692f10)
┣████████████████████┫ [100.00%, 3000/3000, 00:13/00:13, 233.76i/s] (dev = 66.77252f0, tst = (13.354513f0,), mem = 1.1703879f10)


Training Discriminator:


In [52]:
println("Ex.Generated words: ", join(generate(genmodel, MAXLENGTH, 5),"\n"))

Ex.Generated words: lPjjjjj
sssssss
ssssss
ssssss
ssssss
